In [2]:
# Example Notebook file demonstrating how to use the file structure
from utils.preprocess_util_lib_example import save_random_dataframe
from pathlib import Path

save_random_dataframe(Path("../output"), Path("test.csv"))

In [10]:
def determine_comma_role(name: str) -> str:
    """Given a string (someone's name), attempts to determine the role of the
    comma in the name and where it ought to belong.

    Some assumptions are made:
        * If a suffix is included in the name and the name is not just the last
          name(i.e "Doe, Jr), the format is
          (last_name suffix, first and middle name) i.e Doe iv, Jane Elisabeth

        * If a comma is used anywhere else, it is in the format of
          (last_name, first and middle name) i.e Doe, Jane Elisabeth
    Args:
        name: a string representing a name/names of individuals
    Returns:
        the name with or without a comma based on some conditions
    """
    suffixes = [
        "sr",
        "jr",
        "i",
        "ii",
        "iii",
        "iv",
        "v",
        "vi",
        "vii",
        "viii",
        "ix",
        "x",
    ]
    name_parts = name.lower().split(",")
    # if the comma is just in the end as a typo:
    if len(name_parts[1]) == 0:
        return name_parts[0].title()
    # if just the suffix in the end, leave the name as it is
    if name_parts[1].strip() in suffixes:
        return name.title()
    # at this point either it's just poor name placement, or the suffix is
    # in the beginning of the name. Either way, the first part of the list is
    # the true last name.
    last_part = name_parts.pop(0)
    first_part = " ".join(name_parts)
    return first_part.title() + " " + last_part.title()

In [16]:
determine_comma_role("DOe, Jane, Jr")

' Jane  Jr Doe'

In [31]:
def get_likely_name(first_name: str, last_name: str, full_name: str) -> str:
    """Given name related columns, return a person's likely name

    Given different formatting used accross states, errors in data entry
    and missing data, it can be difficult to determine someone's actual
    name. For example, some states have a last name column with values like
    "Doe, Jane", where the person's first name appears to have been erroneously
    included.

    Args:
        first_name: raw value of first name column
        last_name: raw value last name column
        full_name: raw value of name or full_name column
    Returns:
        The most likely full name of the person listed

    Sample Usage:
    >>> get_likely_name("Jane", "Doe", "")
    'Jane Doe'
    >>> get_likely_name("", "", "Jane Doe")
    'Jane Doe'
    >>> get_likely_name("", "Doe, Jane", "")
    'Jane Doe'
    >>> get_likely_name("Jane Doe", "Doe", "Jane Doe")
    'Jane Doe'
    >>> get_likely_name("Jane","","Doe, Sr")
    'Jane Doe, Sr'
    >>> get_likely_name("Jane Elisabeth Doe, IV","Elisabeth","Doe, IV")
    'Jane Elisabeth Doe, Iv'
    >>> get_likely_name("","","Jane Elisabeth Doe, IV")
    'Jane Elisabeth Doe Iv'
    """
    # first ensure clean input by deleting spaces:
    first_name, last_name, full_name = list(
        map(lambda x: x.lower().strip(), [first_name, last_name, full_name])
    )

    # if data is clean:
    if first_name + " " + last_name == full_name:
        return full_name

    # some names have titles or professions associated with the name. We need to
    # remove those from the name.
    titles = [
        "mr",
        "ms",
        "mrs",
        "miss",
        "prof",
        "dr",
        "doctor",
        "sir",
        "madam",
        "professor",
    ]
    names = [first_name, last_name, full_name]

    for i in range(len(names)):
        # if there is a ',' deal with it accordingly
        if "," in names[i]:
            names[i] = determine_comma_role(names[i])

        names[i] = names[i].replace(".", "").split(" ")
        names[i] = [
            name_part for name_part in names[i] if name_part not in titles
        ]
        names[i] = " ".join(names[i])

    # one last check to remove any pieces that might add extra whitespace
    names = list(filter(lambda x: x != "", names))
    names = " ".join(names)
    names = names.title().replace("  "," ").split(" ")
    final_name = []
    [final_name.append(x) for x in names if x not in final_name]
    return " ".join(final_name).strip()

In [36]:
import pandas as pd
ind_df = pd.read_csv("../output/complete_individuals_table.csv")
ind_df.sample(1000)
ind_df

/tmp/ipykernel_4143866/1500712151.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  ind_df = pd.read_csv("../output/complete_individuals_table.csv")


,Unnamed: 0,id,first_name,last_name,full_name,entity_type,state,party,company
0,0,1869727,NaN,NaN,william stoner,individual,NaN,NaN,NaN
1,1,1779679,NaN,NaN,rm coulon,individual,NaN,NaN,area agency on aging
2,2,2277221,NaN,NaN,james engelson,individual,NaN,NaN,retired
3,3,2277156,NaN,NaN,marivic franciaskinner,individual,NaN,NaN,fibre source international corp
4,4,2341373,NaN,NaN,anthony grindle,individual,NaN,NaN,zimmerbiomet
...,...,...,...,...,...,...,...,...,...
2505341,861260,6acfa74b-d5e1-4afd-b020-dbe429eb1c3f,NaN,NaN,Melissa Hart,Candidate,PA,REP,NaN
2505342,861271,f111045d-bc3d-4050-9ad7-b3b1e6d72e56,NaN,NaN,Heather Miller,Candidate,PA,DEM,NaN
2505343,861277,d40859d7-b523-4ef5-895b-c3a947ab582f,NaN,NaN,Christopher M. Gebhard,Candidate,PA,REP,NaN
2505344,861775,f5d76d43-86f4-40f9-aeb9-3df97ca8cdf0,NaN,NaN,April Weaver,Candidate,PA,REP,NaN


In [20]:
determine_comma_role("Doe, Jr, Jane")

'Doe, Jr, Jane'